In [ ]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

REMOVE_LOW_E_IDS = False

def load(dataset):
    disorder = 'Schizophrenia'
    if dataset == 'MCIC':
        path = 'MCIC/'
#         hc_label = 'Control'
#         scz_label = 'Schizophrenia'
        df_demo = pd.read_csv(path + "assessment_data/5720_MCIC_Clinical_Data_20211015.csv").set_index("ID")

    elif dataset == 'COBRE':
        path = 'COBRE/'
#         hc_label = 'No_Known_Disorder'
#         scz_label = 'Schizophrenia_Strict'
        df_demo = pd.read_csv(path + "assessment_data/demographics.csv").set_index("ID")
        df_demo = df_demo.rename({'Gender': 'Sex'}, axis=1)
        df_demo['Sex'] = df_demo['Sex'].replace({1: 'M', 2: 'F'})
        df_demo['group'] = df_demo["group"].replace({"Schizoaffective": "Schizophrenia"})

#         df_demo['group'] = df_demo['group'].replace({"No_Known_Disorder": "Control", "Schizophrenia_Strict": "Schizophrenia"})
        
    elif dataset == "UCLA":
        path = 'UCLA/'
        df_demo = pd.read_csv(path + "participants.tsv", sep="\t").set_index("participant_id")
        df_demo = df_demo.rename(columns={"gender": "Sex", "diagnosis": "group"})
        df_demo['group'] = df_demo["group"].replace({"SCHZ": "Schizophrenia", "CONTROL": "Control"})
        df_demo.index = df_demo.index.str.replace("sub-", "").astype('int')

        
    elif dataset == "CAN-BIND":
        disorder = 'Depression'
        path = 'CAN-BIND/'
        df_demo = pd.read_csv(path + "assessment_data/CBN01_demo.csv").set_index("SUBJLABEL")
        df_demo = df_demo.rename({'SEX': 'Sex', 'Group': 'group', 'AGE': 'age'}, axis=1)
        df_demo['Sex'] = df_demo['Sex'].replace({1: 'F', 2: 'M'})
        df_demo['group'] = df_demo['group'].replace({'Treatment': 'Depression'})
    
    elif dataset == "TOPSY":
        path = 'TOPSY/'
        df_demo = pd.read_csv(path + "subjids_age_sex.csv").set_index("ID")
        df_demo = df_demo.rename({'Gender': 'Sex', 'AgeScan': 'age'}, axis=1)
        df_demo['Sex'] = df_demo['Sex'].replace({1: 'M', 2: 'F'})
#         df_demo['group'] = df_demo['group'].replace({'Treatment': 'Depression'})
    
    
    df_pred_male = pd.read_csv(path + "pred_male.csv").set_index("rownames.df_male.").drop("Unnamed: 0", axis=1)
    df_pred_male = df_pred_male.rename({"pred_male": "pred"}, axis=1)
    df_pred_female = pd.read_csv(path + "pred_female.csv").set_index("rownames.df_female.").drop("Unnamed: 0", axis=1)
    df_pred_female = df_pred_female.rename({"pred_female": "pred"}, axis=1)

    df_pred = pd.concat([df_pred_male, df_pred_female])

#     df_regions = pd.read_csv(path + 'combined.csv').set_index("Unnamed: 0")
    df_shap = pd.read_csv(path + "shap_values.csv").set_index("Unnamed: 0")

    top_cols = df_shap.abs().mean(axis=0).sort_values().index.tolist()[::-1]

    return {
        "shap": df_shap,
        "demo": df_demo,
        "pred": df_pred,
        "top_cols": top_cols,
        "disorder": disorder
#         "regions": df_regions
    }
        
    
COBRE = load('COBRE')
MCIC = load('MCIC')
UCLA = load('UCLA')
CANBIND = load('CAN-BIND')
TOPSY = load('TOPSY')
datasets = {
    "COBRE": COBRE,
    "MCIC": MCIC,
    "UCLA": UCLA,
    "CANBIND": CANBIND,
    "TOPSY": TOPSY
}
CANBIND["pred"].shape, COBRE["pred"].shape, MCIC["pred"].shape, UCLA["pred"].shape, TOPSY["pred"].shape
# of the 197 that I had data for, 4 did not have t1 scans (COBRE) and 1 did not have demographics information
# from the 212, only 203 had t1

# Table 1 information

In [ ]:
import scipy.stats as stats

def age(df_, group=None):
    if group is not None:
        df_ = df_[df_["group"] == group]
    print("Group:", group)
    print("%.2f (%.2f)" % (df_["age"].mean(), df_["age"].std()))
    print("%.2f - %.2f" % (df_["age"].min(), df_["age"].max()))
    
def sex(df_, group=None):
    if group is not None:
        df_ = df_[df_["group"] == group]
    print("Group:", group)
    f = df_["Sex"].value_counts()["F"]
    m = df_["Sex"].value_counts()["M"]
    print("F: %d (%.2f), M: %d (%.2f)" % (f, (f/(f+m))*100, m, (m/(f+m))*100))

for dataset in datasets.keys():
    print("\n\n", dataset)
    print("N")
    disorder = datasets[dataset]['disorder']
    joined = datasets[dataset]["pred"].join(datasets[dataset]["demo"])
    joined = joined[joined.group.isin(["Control", disorder])]
    print(joined["group"].value_counts())
    print("\nAge")
    age(joined, "Control")
    age(joined, disorder)
    age(joined)
    ttest = stats.ttest_ind(joined[joined["group"] == "Control"].age, joined[joined["group"] == disorder].age)
    print("t(%d) = %.2f, p=%.2f" % (joined.shape[0]-1, ttest[0], ttest[1]))
    
    print("\nSex")
    sex(joined, "Control")
    sex(joined, disorder)
    sex(joined)
#     sex_c, sex_s = joined[joined["group"] == "Control"].Sex.value_counts(), joined[joined["group"] == "Schizophrenia"].Sex.value_counts()
#     chi = stats.chisquare([sex_c["F"], sex_c["M"]], [sex_s["F"], sex_s["M"]])
    crosstab = pd.crosstab(joined["group"], joined["Sex"])
    c, p, dof, expected = stats.chi2_contingency(crosstab) 
    print("%.2f (%.2f)" % (c, p))

In [ ]:
# ids = set()
# with open('subjids.txt') as f:
#     for line in f:
#         ids.add(line.rstrip('\n'))
# len(ids)
# ids_= set(COBRE["pred"].index.tolist())
# ids - ids_


# df_shap_regions = datasets["COBRE"]["shap"].join(datasets["COBRE"]["regions"], lsuffix="-shap")

# sns.lmplot(x="TotalGrayVol", y="TotalGrayVol-shap", data=df_shap_regions)
# df_shap_regions

# Gap differences

In [ ]:
import numpy as np
import scipy.stats as stats

if REMOVE_LOW_E_IDS:
    for dataset in datasets.keys():
#         print(low_e_ids[dataset])
        datasets[dataset]["demo"] = datasets[dataset]["demo"].drop(low_e_ids[dataset])
        datasets[dataset]["shap"] = datasets[dataset]["shap"].drop(low_e_ids[dataset])
        datasets[dataset]["pred"] = datasets[dataset]["pred"].drop(low_e_ids[dataset])
        datasets[dataset]["top_cols"] = datasets[dataset]["shap"].abs().mean(axis=0).sort_values().index.tolist()[::-1]

        print('\nRemoved ids')
        print(low_e_ids)

def get_groups(dataset, disorder="Schizophrenia"):
    df_demo = datasets[dataset]["demo"]
    df_pred = datasets[dataset]["pred"].dropna()
    df_demo = df_demo.loc[np.intersect1d(df_demo.index, df_pred.index)]
#     df_demo = df_demo.loc[df_pred.index]
    
    hc_index = df_demo[df_demo.group == "Control"].index
    sz_index = df_demo[df_demo.group == disorder].index

    df_pred_hc = df_pred.loc[hc_index]
    df_pred_sz = df_pred.loc[sz_index]

    gap_hc = df_pred_hc.pred - df_demo.loc[hc_index].age
    gap_sz = df_pred_sz.pred - df_demo.loc[sz_index].age

    age_hc = df_demo.loc[hc_index].age
    pred_hc = df_pred_hc.pred
    
    return gap_hc.dropna(), gap_sz.dropna(), age_hc, pred_hc

def MAPE(gap, age):
    return 100*np.sum(np.abs(gap_hc)/np.abs(age_hc))/len(age_hc)

preds = []
ages = []
rs = []
gap_hc, gap_sz, age_hc, pred_hc = get_groups('COBRE')
preds.append(pred_hc.tolist())
ages.append(age_hc.tolist())
print('COBRE')
print('MAE: ', np.mean(np.abs(gap_hc)))
print('MAPE: ', MAPE(gap_hc, age_hc))
print('r: ', stats.pearsonr(age_hc, pred_hc))
print(stats.ttest_ind(gap_hc, gap_sz, equal_var=False))
low_e_ids_cobre = gap_hc[gap_hc.abs() < 1.0]
rs.append(stats.pearsonr(age_hc, pred_hc))

print('\nMCIC')
gap_hc, gap_sz, age_hc, pred_hc = get_groups('MCIC')
preds.append(pred_hc.tolist())
ages.append(age_hc.tolist())
print('MAE: ', np.mean(np.abs(gap_hc)))
print('MAPE: ', MAPE(gap_hc, age_hc))
print('r: ', stats.pearsonr(age_hc, pred_hc))
print(stats.ttest_ind(gap_hc, gap_sz, equal_var=False))
low_e_ids_mcic = gap_hc[gap_hc.abs() < 1.0]
rs.append(stats.pearsonr(age_hc, pred_hc))

# Maybe I shouldn't do it for UCLA as it wasn't in the training set?
print('\nUCLA')
gap_hc, gap_sz, age_hc, pred_hc = get_groups('UCLA')
preds.append(pred_hc.tolist())
ages.append(age_hc.tolist())
print('MAE: ', np.mean(np.abs(gap_hc)))
print('MAPE: ', MAPE(gap_hc, age_hc))
print('r: ', stats.pearsonr(age_hc, pred_hc))
print(stats.ttest_ind(gap_hc, gap_sz, equal_var=False))
low_e_ids_ucla = []
rs.append(stats.pearsonr(age_hc, pred_hc))

print('\nTOPSY')
gap_hc, gap_sz, age_hc, pred_hc = get_groups('TOPSY')
preds.append(pred_hc.tolist())
ages.append(age_hc.tolist())
print('MAE: ', np.mean(np.abs(gap_hc)))
print('MAPE: ', MAPE(gap_hc, age_hc))
print('r: ', stats.pearsonr(age_hc, pred_hc))
print(stats.ttest_ind(gap_hc, gap_sz, equal_var=False))
rs.append(stats.pearsonr(age_hc, pred_hc))

print('\nCANBIND')
gap_hc, gap_sz, age_hc, pred_hc = get_groups('CANBIND', "Depression")
preds.append(pred_hc.tolist())
ages.append(age_hc.tolist())
print('MAE: ', np.mean(np.abs(gap_hc)))
print('MAPE: ', MAPE(gap_hc, age_hc))
print('r: ', stats.pearsonr(age_hc, pred_hc))
print(stats.ttest_ind(gap_hc, gap_sz, equal_var=False))
rs.append(stats.pearsonr(age_hc, pred_hc))

low_e_ids = {
    "COBRE": low_e_ids_cobre.index.tolist(),
    "MCIC": low_e_ids_mcic.index.tolist(),
    "UCLA": low_e_ids_ucla
}
        
# print("\nPost removal group differences")
# demo_hc = datasets["MCIC"]["demo"]
# demo_hc = demo_hc[demo_hc.group.isin(["Control", "Schizophrenia"])]
# print(stats.ttest_ind(demo_hc[demo_hc["group"] == "Schizophrenia"].age, demo_hc[demo_hc["group"] == "Control"].age))
# print(stats.chi2_contingency(pd.crosstab(demo_hc["group"], demo_hc["Sex"])))
preds_df = pd.DataFrame()
sets = ['COBRE', 'MCIC', 'UCLA', 'TOPSY', 'CANBIND']
for i, pred in enumerate(preds):
    preds_df = pd.concat([
        preds_df,
        pd.DataFrame({"Predicted age": pred, "Source": sets[i], "Chronological age": ages[i]})
    ])

g = sns.lmplot(x="Chronological age", y="Predicted age", hue="Source", data=preds_df, hue_order=[
    "COBRE", "MCIC", "UCLA", "TOPSY", "CANBIND"
], legend=False)

new_text = [
    "COBRE: r(%d)=%.2f; p<0.01" % (len(preds[0])-2, rs[0][0]),
    "MCIC: r(%d)=%.2f; p<0.01" % (len(preds[1])-2, rs[1][0]),
    "UCLA: r(%d)=%.2f; p<0.01" % (len(preds[2])-2, rs[2][0]),
    "TOPSY: r(%d)=%.2f; p=0.36" % (len(preds[3])-2, rs[3][0]),
    "CANBIND: r(%d)=%.2f; p<0.01" % (len(preds[4])-2, rs[4][0])
]
# for t, l in zip(g._legend.texts, new_text):
#     t.set_text(l)
leg = plt.legend(handles=[plt.plot([],ls="-", color=sns.color_palette()[0])[0],
                          plt.plot([],ls="-", color=sns.color_palette()[1])[0],
                          plt.plot([],ls="-", color=sns.color_palette()[2])[0],
                          plt.plot([],ls="-", color=sns.color_palette()[3])[0],
                          plt.plot([],ls="-", color=sns.color_palette()[4])[0]],
    labels=new_text)

# plt.legend()
plt.show()

In [ ]:
# Linear regression of brain-PAD
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels
import matplotlib.pyplot as plt
from statsmodels.compat import lzip
import statsmodels.nonparametric.kernel_regression as kernel_regression
import seaborn as sns


# gap_hc, gap_sz = get_groups('MCIC')
# gap_sz.plot('hist')
dataset = 'CANBIND'
disorder = "Depression"

df_demo = datasets[dataset]["demo"]
df_pred = datasets[dataset]["pred"]
df_join = df_demo[['group', 'age', 'Sex']].join(df_pred)
df_join['gap'] = df_join['pred'] - df_join['age']
df_join = df_join[df_join['group'].isin([disorder, 'Control'])]
df_join = df_join.dropna()
df_join['gaprank'] = df_join['gap'].rank()
df_join['agerank'] = df_join['age'].rank()
reg = smf.ols("gap ~ group + age + Sex", data=df_join).fit()

# reg = kernel_regression.KernelReg("gap ~ group + age + Sex", data=df_join).fit()
# reg = kernel_regression.KernelReg(df_join['gap'], df_join[['age', 'age']], var_type='cc')#.fit()

# pred_val = reg.fittedvalues.copy()
# true_val = df_join['gap'].values.copy()
# residual = true_val - pred_val

# name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
# test = sms.jarque_bera(reg.resid)
# print('Normality of the residuals')
# print(lzip(name, test))


# print('\nLinearity')
# name = ["t value", "p value"]
# test = statsmodels.stats.diagnostic.linear_rainbow(reg)
# print(lzip(name, test))

# # print('Linear', sms.diagnostic.linear_rainbow(reg))
# print('\nHeteroskedasticity')
# name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
# test = sms.het_breuschpagan(reg.resid, reg.model.exog)
# print(lzip(name, test))

# reg.sig_test(0)
# df_join

# fig, ax = plt.subplots(figsize=(6,2.5))
# _ = ax.scatter(residual, pred_val)
reg.summary()
# sns.lmplot(x="age", y="gap", hue="group", data=df_join)

In [ ]:
dataset = "COBRE"
pre_grouped = datasets[dataset]["pred"].join(datasets[dataset]["demo"])
pre_grouped["brain-PAD"] = pre_grouped["pred"] - pre_grouped["age"]

grouped = pre_grouped.groupby("group")
print(grouped.describe()["age"])
print(grouped.describe()["brain-PAD"])
print(grouped.describe(include="object")["Sex"])

# Testing differences in each region

In [ ]:
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
import numpy as np
from collections import defaultdict

name_mapping = {
    'X3rd.Ventricle': '3rd Ventricle (Volume)',
    'rh_R_STSdp_ROI_thickness': 'RH Superior Temporal Sulcus DP (Thickness)',
    'lh_L_STSvp_ROI_thickness': 'LH Superior Temporal Sulcus VP (Thickness)',
    'Right.Lateral.Ventricle': 'Right Lateral Ventricle (Volume)',
    'Right.Thalamus.Proper': 'Right Thalamus Proper (Volume)',
    'Left.Pallidum': 'Left Pallidum (Volume)',
    'Right.Putamen': 'Right Putamen (Volume)',
    'Left.Lateral.Ventricle': 'Left Lateral Ventricle (Volume)',
    'rh_R_STSva_ROI_thickness': 'RH Superior Temporal Sulcus VA (Thickness)',
    'Brain.Stem': 'Brain Stem (Volume)',
    'TotalGrayVol': 'Total Gray Matter Volume'
}
top_n_COBRE = COBRE["top_cols"][0:10]
top_n_MCIC = MCIC["top_cols"][0:10]
top_n_UCLA = UCLA["top_cols"][0:10]
top_n_CANBIND = CANBIND["top_cols"][0:10]
top_n_TOPSY = TOPSY["top_cols"][0:10]
top_n_cols = list(pd.unique(top_n_COBRE + top_n_MCIC + top_n_UCLA + top_n_CANBIND + top_n_TOPSY))
print(top_n_COBRE)
print(top_n_MCIC)
print(top_n_UCLA)
print(top_n_CANBIND)
print(top_n_TOPSY)
print(len(top_n_cols))

run_for = ["COBRE", "MCIC", "UCLA", "TOPSY", "CANBIND"]
results = defaultdict(list)
for col in top_n_cols:
    for dataset in datasets.keys():
        if dataset not in run_for:
            continue
        print(dataset)
        df_shap = datasets[dataset]["shap"]
        df_demo = datasets[dataset]["demo"]
        disorder = datasets[dataset]["disorder"]
        df_joined = df_shap.join(df_demo)
        df_joined = df_joined[df_joined.group.isin(["Control", "%s" % disorder])]
        hc_index = df_demo[df_demo.group == "Control"].index
        sz_index = df_demo[df_demo.group == "%s" % disorder].index
        
        hc_index = hc_index[hc_index.isin(df_shap.index)]
        sz_index = sz_index[sz_index.isin(df_shap.index)]
        
        res = stats.mannwhitneyu(df_shap.loc[hc_index, col].dropna(), df_shap.loc[sz_index, col].dropna())
        reg = smf.ols('Q("%s") ~ group + age + Sex' % col, data=df_joined).fit()
#         print(reg.summary())
#         print(reg.pvalues["group[T.Schizophrenia]"])
#         print(col, res)
#         print(df_shap.loc[hc_index, col].dropna().mean(), df_shap.loc[sz_index, col].dropna().mean())
#         results[col].append(res[1])
        print(reg.pvalues)
        results[col].append(reg.pvalues["group[T.%s]" % disorder].item())
    
#     shap = df_shap[hc_index, col].dropna()
#     age = df_demo[h]
    print()

plt.clf()
results = pd.DataFrame.from_dict(results, orient='index', columns=run_for)
index = results.index.map(name_mapping)
p_values = results.to_numpy()
fdr_res = fdrcorrection(p_values.flatten())

fdr_res = fdr_res[1].reshape((11, len(run_for)))
results = pd.DataFrame(fdr_res, columns=run_for)
results.index = index
cmap = sns.color_palette("magma_r", as_cmap=True)
fig, ax = plt.subplots(figsize=(10,6))         
sns.heatmap(results,
            annot=True, vmin=0.0, vmax=0.05, cmap=cmap, ax=ax)
plt.tight_layout(rect=[0, 0, .95, 1])
plt.savefig('jupyter_results/heatmap.pdf')
plt.savefig('jupyter_results/heatmap.png')
plt.show()
# results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

df_plt_1 = COBRE["shap"].join(COBRE["demo"])
df_plt_1['source'] = 'COBRE'
df_plt_2 = MCIC["shap"].join(MCIC["demo"])
df_plt_2['source'] = 'MCIC'

def annotate(data, **kws):
    r, p = scipy.stats.pearsonr(data['age'], data['shap'])
    if data['group'].all() == 'Schizophrenia':
        offset = 0.05
        name = 'SZ'
    else:
        offset = 0.0
        name = 'HC'
    ax = plt.gca()
    ax.text(.05, 0.05 + offset, '{:s}: r({:d})={:.2f}, p={:.2g}'.format(name, len(data), r, p),
            transform=ax.transAxes)


df_plt = pd.concat([df_plt_1, df_plt_2])
df_plt = df_plt[df_plt.group.isin(['Schizophrenia', 'Control'])]
df_plt = df_plt[top_n_cols + ["source", "group", "age"]]
df_plt = df_plt.melt(["source", "group", "age"], 
                     var_name='region', 
                     value_name='shap')

# g = sns.FacetGrid(df_plt, col="source", row="region", hue="group")

g = sns.lmplot(x='age', y="shap", data=df_plt, hue='group', col='source', row="region")
g.map_dataframe(annotate)
# hist = g.map_dataframe(sns.histplot, x="shap", hue_order=['Control', 'Schizophrenia'],
#                        stat="density", binwidth=0.5, binrange=(-6, 4))
g.add_legend()

#     plt.legend(['Control', 'Schizophrenia'])
g.set(ylabel=col)

plt.show()
#     sns.lmplot(x='age', y='rh_R_STSdp_ROI_thickness', data=df_plt, hue='group')
#     sns.lmplot(x='age', y='lh_L_STSvp_ROI_thickness', data=df_plt, hue='group')
#     sns.lmplot(x='age', y='Right.Lateral.Ventricle', data=df_plt, hue='group')
#     sns.lmplot(x='age', y='Right.Putamen', data=df_plt, hue='group')
#     sns.lmplot(x='age', y='Left.Lateral.Ventricle', data=df_plt, hue='group')
#     sns.lmplot(x='age', y='Right.Thalamus.Proper', data=df_plt, hue='group')
df_plt

# Age-corrected gap and interaction between groups and SHAP

In [ ]:
# maybe I should include site effect?
# Use age correction to find the correlation between each SHAP and the corrected gap
# age_correction
# COBRE["pred"] - COBRE["demo"]["age"]
# COBRE["demo"]["age"]
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

joined_MCIC = MCIC["pred"].join(MCIC["demo"][["age", "group", "Sex", "Site"]])
joined_MCIC = joined_MCIC.rename({"Site": "Scanner"}, axis=1)
joined_MCIC_hc = joined_MCIC[joined_MCIC["group"] == "Control"]
joined_MCIC_hc['gap'] = joined_MCIC_hc['pred'] - joined_MCIC_hc['age']

joined = COBRE["pred"].join(COBRE["demo"][["age", "group", "Sex"]])
joined['Scanner'] = "COBRE"
joined_hc = joined[joined["group"] == "Control"]
joined_hc['gap'] = joined_hc['pred'] - joined_hc['age']

joined_UCLA = UCLA["pred"].join(UCLA["demo"][["age", "group", "Sex", "ScannerSerialNumber"]])
joined_UCLA = joined_UCLA.rename({"ScannerSerialNumber": "Scanner"}, axis=1)
joined_UCLA_hc = joined_UCLA[joined_UCLA["group"] == "Control"]
joined_UCLA_hc['gap'] = joined_UCLA_hc['pred'] - joined_UCLA_hc['age']

joined_CANBIND = CANBIND["pred"].join(CANBIND["demo"][["age", "group", "Sex", "SITESYMBOL"]])
joined_CANBIND = joined_CANBIND.rename({"SITESYMBOL": "Scanner"}, axis=1)
joined_CANBIND_hc = joined_CANBIND[joined_CANBIND["group"] == "Control"]
joined_CANBIND_hc['gap'] = joined_CANBIND_hc['pred'] - joined_CANBIND_hc['age']

joined_TOPSY = TOPSY["pred"].join(TOPSY["demo"][["age", "group", "Sex", "Scanner"]])
joined_TOPSY_hc = joined_TOPSY[joined_TOPSY["group"] == "Control"]
joined_TOPSY_hc['gap'] = joined_TOPSY_hc['pred'] - joined_TOPSY_hc['age']

# sns.lmplot(x="age", y="gap", data=joined_hc)

def reg(x, y):
    x = np.array(x).T
    x = sm.add_constant(x)
    return sm.OLS(endog=y, exog=x).fit()

r = reg(joined_hc["age"], joined_hc["gap"])
offset = r.params[1]*joined["age"] + r.params[0]
joined["cpred"] = joined["pred"] - offset
joined["cgap"] = joined["cpred"] - joined["age"]
joined["gap"] = joined["pred"] - joined["age"]

r = reg(joined_MCIC_hc["age"], joined_MCIC_hc["gap"])
offset = r.params[1]*joined_MCIC["age"] + r.params[0]
joined_MCIC["cpred"] = joined_MCIC["pred"] - offset
joined_MCIC["cgap"] = joined_MCIC["cpred"] - joined_MCIC["age"]

r = reg(joined_UCLA_hc["age"], joined_UCLA_hc["gap"])
offset = r.params[1]*joined_UCLA["age"] + r.params[0]
joined_UCLA["cpred"] = joined_UCLA["pred"] - offset
joined_UCLA["cgap"] = joined_UCLA["cpred"] - joined_UCLA["age"]

r = reg(joined_CANBIND_hc["age"], joined_CANBIND_hc["gap"])
offset = r.params[1]*joined_CANBIND["age"] + r.params[0]
joined_CANBIND["cpred"] = joined_CANBIND["pred"] - offset
joined_CANBIND["cgap"] = joined_CANBIND["cpred"] - joined_CANBIND["age"]

r = reg(joined_TOPSY_hc["age"], joined_TOPSY_hc["gap"])
offset = r.params[1]*joined_TOPSY["age"] + r.params[0]
joined_TOPSY["cpred"] = joined_TOPSY["pred"] - offset
joined_TOPSY["cgap"] = joined_TOPSY["cpred"] - joined_TOPSY["age"]

joined = joined[joined.group.isin(["Control", "Schizophrenia"])]
joined_MCIC = joined_MCIC[joined_MCIC.group.isin(["Control", "Schizophrenia"])]
joined_UCLA = joined_UCLA[joined_UCLA.group.isin(["Control", "Schizophrenia"])]
joined_CANBIND = joined_CANBIND[joined_CANBIND.group.isin(["Control", "Depression"])]
joined_TOPSY = joined_TOPSY[joined_TOPSY.group.isin(["Control", "Schizophrenia"])]

# sns.lmplot(x="age", y="gap", hue="group", data=joined)
# plt.show()
# sns.lmplot(x="age", y="cgap", hue="group", data=joined)
# plt.show()


joined = joined.join(COBRE["shap"])
joined_MCIC = joined_MCIC.join(MCIC["shap"])
joined_UCLA = joined_UCLA.join(UCLA["shap"])
joined_CANBIND = joined_CANBIND.join(CANBIND["shap"])
joined_TOPSY = joined_TOPSY.join(TOPSY["shap"])

joined['source'] = 'COBRE'
joined_MCIC['source'] = 'MCIC'
joined_UCLA['source'] = 'UCLA'
# joined_CANBIND['source'] = joined_CANBIND['Site']
joined_TOPSY['source'] = 'TOPSY'

# print(joined_MCIC)

def run_analysis(joined_f, disorder, dataset=''):
    plt.clf()
    df_summaries = []
    rs = []

    for col in top_n_cols:
    #     print(joined)
        r = smf.rlm(formula='cgap ~ group*Q("%s") + age + Sex + Scanner' % col, data=joined_f).fit()
    #     r = smf.rlm(formula='cgap ~ group*Q("%s") + source + age + Sex' % col, data=joined_f).fit()
        rs.append(r)
        deg_res = r.df_resid
        df_summary = r.summary()#.tables[1]
        print(df_summary)
        results_as_html = df_summary.tables[1].as_html()
        df_summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
        
        df_summary['source'] = col
        df_summaries.append(df_summary)

    #     pred_val = r.fittedvalues.copy()
    #     print(pred_val.shape)
    #     true_val = joined_f['cgap'].values.copy()
    #     print(true_val.shape)
    #     residual = true_val - pred_val

    #     name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
    #     test = sms.jarque_bera(r.resid)
    #     print('Normality of the residuals')
    #     print(lzip(name, test))


    # #     print('Linearity')
    # #     name = ["t value", "p value"]
    # #     test = sms.linear_harvey_collier(r)
    # #     print(lzip(name, test))

    #     print('Linear', sms.diagnostic.linear_rainbow(r))
    #     print('Homoscedasticity', sms.het_breuschpagan(r.resid, r.model.exog))
    #     break
    #     print(col, res)

    print(joined_f.shape)
    
    interaction_value = pd.concat(df_summaries).loc['group[T.%s]:Q("TotalGrayVol")' % disorder]
    p_interaction = interaction_value['P>|z|']
    print(interaction_value)
    sns.lmplot(x="TotalGrayVol", hue="group", y="cgap",
               hue_order=["Control", disorder], data=joined_f, legend=False, order=1)
    # sns.residplot(x="TotalGrayVol", y="cgap", data=joined_f, order=1)
    r1 = stats.pearsonr(joined_f[joined_f.group == "Control"]["TotalGrayVol"], joined_f[joined_f.group == "Control"]["cgap"])
    p_r1 = r1[1]
    
    r2 = stats.pearsonr(joined_f[joined_f.group == "%s" % disorder]["TotalGrayVol"], joined_f[joined_f.group == "%s" % disorder]["cgap"])
    p_r2 = r2[1]
    
    ps = [p_r1, p_r2, p_interaction]
    p_texts = []
    for p in ps:
        if p < 0.01:
            p_text = 'p<0.01'
        else:
            p_text = 'p=%.2f' % p
        p_texts.append(p_text)
    
    leg = plt.legend(handles=[plt.plot([],ls="-", color=sns.color_palette()[0])[0],
                              plt.plot([],ls="-", color=sns.color_palette()[1])[0],
                              plt.plot([], ls="-", color='white')[0]],
        labels=["Control: r(%d)=%.2f; %s" % (len(joined_f[joined_f.group == "Control"]), r1[0], p_texts[0]),
                "%s: r(%d)=%.2f; %s" % (disorder, len(joined_f[joined_f.group == "%s" % disorder]), r2[0], p_texts[1]),
                "Group and SHAP interaction: r(%d)=%.2f; %s" % (deg_res, np.round(interaction_value['coef'], 2), p_texts[2])])
#     LH = leg.legendHandles
#     plt.legend(handles = [plt.plot([],ls="-", color=sns.color_palette()[0])[0]],
#                labels=['aux'])
#     LH[0].set_linewidth(8)  # Works
#     LH[1].set_color('y') # No change
#     LH[2].set_ms(15)    # No change
#     LH[3].set_marker('*')  # Stars added to left and right!
#     LH[3].set_color('r')   # Stars now change color
#     LH[4].set_color('w')   # No change
#     LH[5].set_alpha(0.5)  # Well-behaved scatter

    plt.ylabel('Brain age gap')
    plt.xlabel('Total Gray Matter Volume SHAP value')
    plt.title(dataset)
    plt.tight_layout()
    name = 'jupyter_results/interaction-totalgrayvol-%s' % disorder
    if dataset != '':
        name += '-' + dataset
    plt.savefig('%s.pdf' % name)
    plt.savefig('%s.png' % name)
    plt.show()

    plt.clf()

#     sns.lmplot(x="Right.Lateral.Ventricle", hue="group", y="cgap", data=joined_f, legend=False, order=1)
#     # sns.residplot(x="TotalGrayVol", y="cgap", data=joined_f, order=1)
#     r1 = stats.pearsonr(joined_f[joined_f.group == "Control"]["Right.Lateral.Ventricle"], joined_f[joined_f.group == "Control"]["cgap"])
#     r2 = stats.pearsonr(joined_f[joined_f.group == "%s" % disorder]["Right.Lateral.Ventricle"], joined_f[joined_f.group == "%s" % disorder]["cgap"])
# #     plt.legend(["Control: r(%d)=%.2f; p=%.2f" % (len(joined_f[joined_f.group == "Control"]), r1[0], r1[1]),
# #                 "%s: r(%d)=%.2f; p=%.2f" % (disorder, len(joined_f[joined_f.group == "%s" % disorder]), r2[0], r2[1])])
#     leg = plt.legend(handles=[plt.plot([],ls="-", color=sns.color_palette()[0])[0], plt.plot([],ls="-", color=sns.color_palette()[1])[0]],
#         labels=["Control: r(%d)=%.2f; p=%.2f" % (len(joined_f[joined_f.group == "Control"]), r1[0], r1[1]),
#                 "%s: r(%d)=%.2f; p=%.2f" % (disorder, len(joined_f[joined_f.group == "%s" % disorder]), r2[0], r2[1])])
#     plt.ylabel('Brain age gap')
#     plt.xlabel('Right Lateral Ventricle Volume SHAP value')
#     plt.tight_layout()
#     plt.savefig('jupyter_results/interaction-Right.Lateral.Ventricle-%s.pdf' % disorder)
#     plt.savefig('jupyter_results/interaction-Right.Lateral.Ventricle-%s.png' % disorder)

    # print(r1, r2)
    # joined_f["group"].value_counts()
    # r.pvalues
#     joined_f
    pd.concat(df_summaries).to_csv('jupyter_results/interaction_results-%s.csv' % disorder)

    names = [r_.pvalues.index[-2] for r_ in rs]
    i_pvalues = [r_.pvalues[-2] for r_ in rs]
    print(list(zip(names, fdrcorrection(i_pvalues)[1])))
    # r = smf.ols(formula='cgap ~ group*Q("%s") + group*Q("%s") + source + age + Sex' % ("TotalGrayVol", "Right.Putamen"), data=joined_f).fit()
    # df_summary = r.summary()#.tables[1]
    # df_summary

joined_f = pd.concat([joined, joined_MCIC, joined_UCLA, joined_TOPSY])
joined_f = joined_f[["cgap", "group", "source", "age", "Sex", "Scanner"] + top_n_cols]

# print(joined_f)
# joined_f = joined_f.dropna()
run_analysis(joined_f, 'Schizophrenia')
plt.show()
# run_analysis(joined_MCIC[["cgap", "group", "source", "age", "Sex", "Scanner"] + top_n_cols],
#              'Schizophrenia', 'MCIC')
run_analysis(joined_CANBIND[["cgap", "group", "age", "Sex", "Scanner"] + top_n_cols].dropna(), 'Depression')
plt.show()

In [ ]:
# rs[-1].summary()
joined_f